In [2]:
!pip install scipy
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Read in the main df and group it by county taking the median of all columns in order to have an average for 
#PM2.5 over all years that is not impacted by extreme values
df = pd.read_csv('main_df.csv')
dfg = df.groupby('County').median()

## T-test

In [24]:
#Take the top 25% and bottom 25% counties in terms of PM2.5 levels  
q75 = dfg.loc[:, 'Average PM 2.5/County'].quantile(.75)
q25 = dfg.loc[:, 'Average PM 2.5/County'].quantile(.25)

dfLn = list(dfg[dfg['Average PM 2.5/County'] <= q25].index)
dfHn = list(dfg[dfg['Average PM 2.5/County'] >= q75].index)

dfL = df[df['County'].isin(dfLn)]
dfH = df[df['County'].isin(dfHn)]

dfL = dfL[dfL['Data Year'] == 2022]
dfH = dfH[dfH['Data Year'] == 2022]

In [30]:
#Find the EV adoption rate of the counties chosen above
EVL = dfL.loc[:, 'Number of EV']/(dfL.loc[:, 'Number of EV']+dfL.loc[:, 'Number of Gas Cars'])
EVH = dfH.loc[:, 'Number of EV']/(dfH.loc[:, 'Number of EV']+dfH.loc[:, 'Number of Gas Cars'])

#Conduct a Welch's t-test for independence since the variance between counties is not identical
pvalue = stats.ttest_ind(EVL, EVH, equal_var = False)[1]
print('The p-value is: ', pvalue)

The p-value is:  0.011677101262940925


In [ ]:
# Violin Plot
#Create matplot figure and two subplot axes that share y
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 8), sharey=True)

#Create first violinplot of low air pollutin counties EV adoption rate
parts1 = ax1.violinplot(EVL)
ax1.set_title('Counties with Low Air Pollution EV Adoption Rates')
ax1.set_ylabel('EV Adoption Rate')

#Loop through all facecolors and set them equal to blue
for pc in parts1['bodies']:
    pc.set_facecolor('blue')

#Create second violinplot of high air pollutin counties EV adoption rate
parts2 = ax2.violinplot(EVH)
ax2.set_title('Counties with High Air Pollution EV Adoption Rates')

#Loop through all facecolors and set them equal to red
for pc in parts2['bodies']:
    pc.set_facecolor('red')

#Clean up figure and save
plt.tight_layout(pad=5)
plt.savefig('Violin.png')
